In [0]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.utils import np_utils

import os,keras
import tensorflow as tf

In [0]:
path_to_learning_set = "/content/gdrive/My Drive/MIDI_files"
path_to_note_file = "/content/gdrive/My Drive/Notes"
path_to_model = "/content/gdrive/My Drive/Model.h5"

In [0]:
learning_rate = 0.002
epochs_number = 150

In [91]:
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  tpu_adress = 'grpc://' + device_name
  print('TPU found at: {}'.format(tpu_adress))

except KeyError:
  print('No TPU found. Chill. We can fix it.')

TPU found at: grpc://10.0.230.218:8470


In [92]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [93]:
notes = []

for plik in glob.glob(path_to_learning_set + "/test/*.mid"):
    midi = converter.parse(plik) # parsing midi to notes and chords

    print('Parsing:', plik)

    notes_to_parse = None

    # for midi with many instruments
    try:
        part = instrument.partitionByInstrument(midi)
        notes_to_parse = part.parts[0].recurse() 
    except:
        notes_to_parse = midi.flat.notes

    for sound in notes_to_parse:
        if isinstance(sound, note.Note): # if it is one note
            notes.append(str(sound.pitch))
        elif isinstance(sound, chord.Chord): # if it is chord, save notes seperated by dot
            notes.append('.'.join(str(n) for n in sound.normalOrder))

# Parse songs from midi to strings
with open(path_to_note_file, 'wb') as path:
    pickle.dump(notes, path)

Parsing: /content/gdrive/My Drive/MIDI_files/test/minuet-of-woods.mid
Parsing: /content/gdrive/My Drive/MIDI_files/test/nocturne-of-shadows.mid


In [0]:
sequence_length = 100

# all sounds names
sounds_names = sorted(set(notes))

number_of_sounds = len(sounds_names) # number of possible sounds

# maping
notes_in_int = dict((n, i) for i, n in enumerate(sounds_names))

network_input = []
network_output = []

for i in range(0, len(notes) - number_of_sounds):
    sequence_input = notes[i:i + number_of_sounds] # we take this many -> number_of_sounds, notes and put them in sequence_input
    sequence_output = notes[i + number_of_sounds] # and one sound that is after that sequence
    network_input.append([notes_in_int[n] for n in sequence_input]) # to input we put int representation of sequance
    network_output.append(notes_in_int[sequence_output]) # to output we put int represnation of next note

number_of_patterns = len(network_input) # how many records we have, every record is sequence of sounds + one sound after that sequnce

network_input = np.reshape(network_input, (number_of_patterns, number_of_sounds, 1))
network_input = network_input / float(number_of_sounds)

network_output = np_utils.to_categorical(network_output)

In [95]:
tf.config.experimental_connect_to_host(tpu_adress)
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_adress)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

with strategy.scope():
  # creating optimizer
  opt = tf.compat.v1.train.RMSPropOptimizer(learning_rate)
 
        

  input_layer=tf.keras.layers.Input(shape=(network_input.shape[1], network_input.shape[2]))
  x=tf.keras.layers.LSTM(512, return_sequences=True)(input_layer)
  x=tf.keras.layers.Dropout(0.3)(x)
  x=tf.keras.layers.LSTM(512, return_sequences=True)(x)
  x=tf.keras.layers.Dropout(0.3)(x)
  x=tf.keras.layers.LSTM(512)(x)
  x=tf.keras.layers.Dense(256)(x)
  x=tf.keras.layers.Dropout(0.3)(x)

  # output layer must have as many neurones as possible notes in our program
  output_layer = tf.keras.layers.Dense(number_of_sounds, activation='softmax')(x)
  model_f=tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
  model_f.compile(loss='categorical_crossentropy', optimizer=opt)  # kompilacja modelu

  
# konwersja modelu biblioteki Keras na model TensorFlow przystosowany do pracy z TPU


INFO:tensorflow:Initializing the TPU system: grpc://10.0.230.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.0.230.218:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [0]:
# saving better moodels
callback_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    path_to_model,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

In [97]:
# newtork training
model_f.fit(network_input, network_output, epochs=epochs_number, batch_size=1024, callbacks=[callback_checkpoint])

Epoch 1/150
1/1 [==============================] - ETA: 0s - loss: 2.6367WARNING:tensorflow:TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


1/1 [==============================] - 1s 1s/step - loss: 2.6367
Epoch 2/150
1/1 [==============================] - ETA: 0s - loss: 2.6351WARNING:tensorflow:TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


1/1 [==============================] - 0s 440ms/step - loss: 2.6351
Epoch 3/150
1/1 [==============================] - ETA: 0s - loss: 2.6330WARNING:tensorflow:TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


1/1 [==============================] - 1s 611ms/step - loss: 2.6330
Epoch 4/150
1/1 [==============================] - 0s 5ms/step - loss: 2.6349
Epoch 5/150
1/1 [==============================] - ETA: 0s - loss: 2.6329WARNING:tensorflow:TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


1/1 [==============================] - 0s 393ms/step - loss: 2.6329
Epoch 6/150
1/1 [==============================] - ETA: 0s - loss: 2.6296WARNING:tensorflow:TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


...

1/1 [==============================] - 0s 463ms/step - loss: 1.8978
Epoch 81/150
1/1 [==============================] - 0s 6ms/step - loss: 1.9365
Epoch 82/150
1/1 [==============================] - 0s 5ms/step - loss: 1.9157
Epoch 83/150
1/1 [==============================] - 0s 6ms/step - loss: 1.9263
Epoch 84/150
1/1 [==============================] - ETA: 0s - loss: 1.8692WARNING:tensorflow:TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


1/1 [==============================] - 0s 222ms/step - loss: 1.8692
Epoch 85/150
1/1 [==============================] - 0s 7ms/step - loss: 1.8755
Epoch 86/150
1/1 [==============================] - ETA: 0s - loss: 1.7910WARNING:tensorflow:TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


1/1 [==============================] - 1s 718ms/step - loss: 1.7910
Epoch 87/150
1/1 [==============================] - ETA: 0s - loss: 1.7335WARNING:tensorflow:TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file. You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).


1/1 [==============================] - 0s 174ms/step - loss: 1.7335
Epoch 88/150
1/1 [==============================] - 0s 4ms/step - loss: 1.8963
Epoch 89/150
1/1 [==============================] - 0s 5ms/step - loss: 1.9064
Epoch 90/150
1/1 [==============================] - 0s 7ms/step - loss: 1.8411
Epoch 91/150
1/1 [==============================] - 0s 6ms/step - loss: 1.8995
Epoch 92/150
1/1 [==============================] - 0s 4ms/step - loss: 1.9165
Epoch 93/150
1/1 [==============================] - 0s 4ms/step - loss: 1.7722
Epoch 94/150
1/1 [==============================] - 0s 5ms/step - loss: 1.8695
Epoch 95/150
1/1 [==============================] - 0s 6ms/step - loss: 1.7642
Epoch 96/150
1/1 [==============================] - 0s 4ms/step - loss: 1.8188
Epoch 97/150
1/1 [==============================] - 0s 4ms/step - loss: 1.8938
Epoch 98/150
1/1 [==============================] - 0s 5ms/step - loss: 1.9704
Epoch 99/150
1/1 [==============================] - 0s 6ms/step

1/1 [==============================] - 0s 186ms/step - loss: 1.7122
Epoch 121/150
1/1 [==============================] - 0s 4ms/step - loss: 2.1124
Epoch 122/150
1/1 [==============================] - 0s 6ms/step - loss: 2.0573
Epoch 123/150
1/1 [==============================] - 0s 3ms/step - loss: 1.9690
Epoch 124/150
1/1 [==============================] - 0s 5ms/step - loss: 1.8860
Epoch 125/150
1/1 [==============================] - 0s 9ms/step - loss: 1.7968
Epoch 126/150
1/1 [==============================] - 0s 5ms/step - loss: 1.8912
Epoch 127/150
1/1 [==============================] - 0s 6ms/step - loss: 1.8028
Epoch 128/150
1/1 [==============================] - 0s 3ms/step - loss: 1.8289
Epoch 129/150
1/1 [==============================] - 0s 4ms/step - loss: 1.8499
Epoch 130/150
1/1 [==============================] - 0s 5ms/step - loss: 1.9961
Epoch 131/150
1/1 [==============================] - 0s 7ms/step - loss: 1.7691
Epoch 132/150
1/1 [==============================] -